In [45]:
import requests
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import os
import gradio as gr
import sys
import json

print("Python:", sys.executable)
print("Gradio version:", gr.__version__)
print("Gradio loaded from:", gr.__file__)

Python: C:\Users\Arnav Karkhanis\workspace\personal-agent-ideas\agent-ideas\.venv\Scripts\python.exe
Gradio version: 6.0.2
Gradio loaded from: C:\Users\Arnav Karkhanis\workspace\personal-agent-ideas\agent-ideas\.venv\Lib\site-packages\gradio\__init__.py


In [46]:
load_dotenv(override=True)
openai = OpenAI()
perplexity_api_key = os.getenv('PERPLEXITY_API_KEY')
pushover_user = os.getenv('PUSHOVER_USER')
pushover_token = os.getenv('PUSHOVER_TOKEN')
pushover_url = "https://api.pushover.net/1/messages.json"

print(f"pushover user: {pushover_user}")

if perplexity_api_key:
    print(f"Perplexity API Key exists and begins {perplexity_api_key[:4]}")
else:
    print("Perplexity API Key not set (and this is optional)")

perplexity = OpenAI(api_key=perplexity_api_key, base_url="https://api.perplexity.ai")

model_name = "sonar"
openai_model_name = "gpt-5-nano"

pushover user: u5r8tbhxpogy2j89zcf14zyz45igj6
Perplexity API Key exists and begins pplx


In [47]:
def push(message):
    print(f"Push message: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)



In [5]:
push("Hey!!")

Push message: Hey!!


In [48]:
def record_user_details(email, name="Name not provided", notes="Not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [49]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I could not answer")
    return {"recorded": "ok"}

In [50]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [51]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [52]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [36]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['quest

In [53]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        tool = globals().get(tool_name)
        result = tool(**arguments)
        # if tool_name == "record_user_details":
        #     result = record_user_details(**arguments)
        # elif tool_name == "record_unknown_question":
        #     result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [38]:
globals()["record_unknown_question"]("This is a tough question")

Push message: Recording This is a tough question asked that I could not answer


{'recorded': 'ok'}

In [54]:
reader = PdfReader("../resources/profile/LinkedIn-Profile.pdf")
linkedInText = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedInText = linkedInText + text

reader = PdfReader("../resources/profile/Vaibhav_Karkhanis_Executive_Director_Resume.pdf")
resumeText = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resumeText = resumeText + text

In [122]:
# linkedInText
# resumeText

In [55]:
with open("../resources/profile/AI-generated-professional-summary.txt", "r", encoding="utf-8") as f:
    summary = f.read();

In [124]:
# summary

In [56]:
name = "Vaibhav Karkhanis"

In [65]:
system_prompt = f"You are acting as {name}. You are answering on behalf of {name} on his website \
particularly questions related to {name}'s career, background, skills, interests and experience. \
You can use combination of LinkedinProfile and AI-generated-professional-summary.txt to answer questions \
Be uttermost professional and engaging because most of the users may be recruiters from highly acclaimed \
firms and you should talk like you are talking to future employer or potential client for any consulting work or Full time employee role. If the role is not consulting and if it is full time equivalent then I am looking for a senior position equivalent to Executive Director. If the questions are related to the compensation and pay package or CTC, please ask the user to contact me directly using the contact information mentioned. Kindly ensure the genuinity of the person interacting before revealing any information like by asking the consultancy name or recruiter name and verifying the information to insulate from a  potential fraud. If you do not know an answer, deny replying and provide user with the contact information. This is non-negotiable. Do not answer questions which are not provided in the documents given and do not assume, instead ask the user to contact me by provided the given contact information. Be thoroughly professional and this is non-negotiable. Also if the questions are irrelevant and on any other subject other than profile please clearly deny to answer and ask the user to stick to relevant questions about profile only but deny in professional tone. Please be strict on this. If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career.If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool."

system_prompt += f"\n\n## Summary: \n{summary}\n\n## LinkedIn Profile:\n\n{linkedInText}\n\n## Resume:\n\n{resumeText}\n\n"
system_prompt += f"\n\nWith this prompt please chat with the user always maintaining the character as {name}"


In [66]:
# system_prompt

In [67]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        openai_response = openai.chat.completions.create(model=openai_model_name, messages=messages, tools=tools)
        finish_reason = openai_response.choices[0].finish_reason
        if finish_reason == "tool_calls":
            message = openai_response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    # response = perplexity.chat.completions.create(model=model_name, messages=messages)
    return openai_response.choices[0].message.content

In [68]:
gr.ChatInterface(chat,analytics_enabled=False).launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Tool called: record_unknown_question
Push message: Recording Who is your professional role model asked that I could not answer


In [130]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [131]:
evaluator_system_prompt = f"You are an evaluator that decides whether response to a question is acceptable. \
You are provided with conversation between User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## System prompt fed to the agent is: \n{system_prompt}\n\n based on which the agent is supposed to answer"

evaluator_system_prompt += f"\n\n## Summary: \n{summary}\n\n## LinkedIn Profile:\n\n{linkedInText}\n\n## Resume:\n\n{resumeText}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [132]:
def evaluator_user_prompt(agent_reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{agent_reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [133]:
def evaluate(eval_reply, message, history)-> Evaluation:
    eval_message = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(eval_reply, message, history)}]
    perp_res = perplexity.chat.completions.parse(model=model_name, messages=eval_message, response_format=Evaluation)
    return perp_res.choices[0].message.parsed

In [134]:
test_messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "what is spiritual conquest"}]
test_response = openai.chat.completions.create(model=openai_model_name, messages=test_messages)
test_reply = test_response.choices[0].message.content

In [135]:
test_reply

In [136]:
evaluate(test_reply, test_messages, test_messages[:1])

In [137]:
def rerun(rerun_reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{rerun_reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    openai_rerun_res = openai.chat.completions.create(model=openai_model_name, messages=messages)
    return openai_rerun_res.choices[0].message.content

In [138]:
def chat(user_message, history):
    first_prompt = system_prompt
    messages_for_first_chat =[{"role": "system", "content": first_prompt}] + history + [{"role": "user", "content": user_message}]
    res_from_first_chat = openai.chat.completions.create(model=openai_model_name, messages=messages_for_first_chat)
    reply_from_first_chat =res_from_first_chat.choices[0].message.content

    # Evaluate the response
    evaluation = evaluate(reply_from_first_chat, user_message, history)
    print(evaluation)
    if evaluation.is_acceptable:
        print("Response approved")
    else:
        print("Response rejected - This response needs to be reconsidered")
        reply_from_first_chat = rerun(reply_from_first_chat, user_message, history, evaluation.feedback)
    return reply_from_first_chat

In [139]:
gr.ChatInterface(chat,analytics_enabled=False).launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.
